# Generar audio con Python

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 15 de abril de 2025


## Funciones auxiliares

Para generar audio se usarán sumas de cosenos.

Al sumar ondas es posible superar el rango [-1, 1], se puede corregir de dos formas:
  * Normalizar: que consiste en reducir la amplitud (baja el volumen) hasta dejar el máximo y mínimo dentro de [-1, 1].
  * Saturar: la onda se trunca cuando supera 1 o -1. Mantiene las amplitudes originales, pero el sonido no se escucha bien.

Se usará normalización, pero se puede cambiar el código para probar como se escucha con saturación.

In [ ]:
import math
import numpy
import os
import IPython.display as ipd    
import matplotlib.pyplot as plt
%matplotlib inline

# genera una onda coseno
def generar_onda(ciclos_por_segundo, num_samples, sample_rate):
    samples = numpy.zeros(num_samples, numpy.float32)
    # este offset evita que todas las ondas comiencen en su valor máximo
    offset = sample_rate / ciclos_por_segundo / 4
    for i in range(num_samples):
        samples[i] = (i + offset) * (ciclos_por_segundo * 2 * math.pi) / sample_rate
    samples = numpy.cos(samples)
    return samples


def generar_audio(largo_segundos, sample_rate, ondas, normalizar=True):
    num_samples = round(largo_segundos * sample_rate)
    audio_samples = numpy.zeros(num_samples, numpy.float32)
    for onda in ondas:
        ciclos_por_segundo, amplitud = onda
        assert ciclos_por_segundo < sample_rate / 2
        onda_samples = generar_onda(ciclos_por_segundo, num_samples, sample_rate)
        audio_samples += amplitud * onda_samples
    # normalizar al rango [-1, 1]
    if normalizar:
        maximo = max(numpy.max(audio_samples), abs(numpy.min(audio_samples)))
        if maximo > 1:
            audio_samples /= maximo
    # saturar al rango [-1, 1] 
    else:
        audio_samples[audio_samples > 1] = 1
        audio_samples[audio_samples < -1] = -1
    return audio_samples


#funcion que dibuja un array de samples
def mostrar_samples(samples_array, titulo):
    plt.figure(figsize=(25,5))
    plt.title(titulo, {"fontsize":32})
    plt.plot(samples_array)
    plt.xlabel('Samples')
    plt.ylabel('Amplitud')
    plt.ylim(top=1, bottom=-1)
    plt.yticks(numpy.arange(-1, 1, step=0.2))
    plt.margins(0)
    plt.show()


def guardar_samples_raw(audio_samples, sample_rate, filename):
    samples_16bits = (audio_samples * 32767).astype(numpy.int16)
    carpeta_temporal = "audios_temporales"
    os.makedirs(carpeta_temporal, exist_ok=True)
    archivo = "{}/{}.{}_{}.raw".format(carpeta_temporal, filename, sample_rate, "s16le")
    print("creando {}".format(archivo))
    samples_16bits.tofile(archivo)



## 1-Generar onda sumando cosenos

Vamos a sumar ondas de distinta frecuencia y con distinta amplitud. Las amplitudes se normalizarán al rango [-1, 1].

Se debe ingresar un `sample_rate` del audio generado (22050 o 44100 funciona bien).  

El `sample_rate` impone un límite a la frecuencia máxima que se puede usar.



In [ ]:
sample_rate = 44100
largo_onda_segundos = 1

ondas1 =  [(200, 1), (1800, 0.25), (4500, 0.25)]
samples1 = generar_audio(largo_onda_segundos, sample_rate, ondas1)

ondas2 =  [(500, 1), (2200, 1), (3100, 0.25)]
samples2 = generar_audio(largo_onda_segundos, sample_rate, ondas2)

ondas3 =  [(300, 0.3), (2400, 1), (5000, 0.25)]
samples3 = generar_audio(largo_onda_segundos, sample_rate, ondas3)

#solo se muestran los primeros 30 milisegundos (para ver la forma del audio)
largo_mostrar = int(sample_rate * 0.03)

mostrar_samples(samples1[0:largo_mostrar], "ondas1")
mostrar_samples(samples2[0:largo_mostrar], "ondas2")
mostrar_samples(samples3[0:largo_mostrar], "ondas3")

samples = numpy.concatenate((samples1, samples2, samples3))

# samples debe estar en el rango [-1, 1]
ipd.Audio(samples, rate=sample_rate, autoplay=True, normalize=False)


## 2-Generar una escala

Vamos a generar una escala de frecuencias, similar a lo que se puede oir en https://www.youtube.com/watch?v=qNf9nzvnd1k

Se mostrarán las frecuencias en forma lineal (con un paso fijo). Esto implica que al principio hay gran cambio (cambia bastante el sonido al cambviar de 100Hz a 150Hz) pero al final no se notará cambio (casi no cambia el sonido entre de 10000Hz a 10050Hz)

In [ ]:
def generar_escala(ffrecuencias, segundos_por_ventana, sample_rate):
    num_ventanas = len(frecuencias)
    samples_por_ventana = round(segundos_por_ventana * sample_rate)
    num_samples = num_ventanas * samples_por_ventana
    audio_samples = numpy.zeros(num_samples, numpy.float32)
    print("num_ventanas={} largo_total={:.1f} seg".format(num_ventanas, num_samples/sample_rate))
    inicio = 0
    for frecuencia in frecuencias:
        # se reusa la función anterior de generar ondas
        onda_samples = generar_onda(frecuencia, samples_por_ventana, sample_rate)
        audio_samples[inicio : inicio + samples_por_ventana] = onda_samples[:]
        inicio += samples_por_ventana
    # no es necesario normalizar ni saturar porque no se estan sumando ondas, siempre estará en [-1,1]
    return audio_samples

frecuencia_inicio = 100
frecuencia_fin = 22000
frecuencia_paso = 50

segundos_por_ventana = 0.1
sample_rate = 44100

# bajar un poco el volumen global, para que no duelan los oidos :-)
volumen_global = 0.2

# se generan las frecuencias de cada ventana en forma lineal
frecuencias = range(frecuencia_inicio, frecuencia_fin, frecuencia_paso)

#generar los samples para cada frecuencia
samples = generar_escala(frecuencias, segundos_por_ventana, sample_rate)

#bajar el volumen globalmente
samples *= volumen_global

largo_mostrar = int(sample_rate * 0.02)

inicio_1 = int(sample_rate * 0)
inicio_2 = int(sample_rate * 2)
inicio_3 = int(sample_rate * 4)
inicio_4 = int(sample_rate * 8)
inicio_5 = int(sample_rate * 16)

mostrar_samples(samples[inicio_1:inicio_1+largo_mostrar], "frecuencia 1")
mostrar_samples(samples[inicio_2:inicio_2+largo_mostrar], "frecuencia 2")
mostrar_samples(samples[inicio_3:inicio_3+largo_mostrar], "frecuencia 3")
mostrar_samples(samples[inicio_4:inicio_4+largo_mostrar], "frecuencia 4")
mostrar_samples(samples[inicio_5:inicio_5+largo_mostrar], "frecuencia 5")

guardar_samples_raw(samples, sample_rate, "escala")
ipd.Audio(samples, rate=sample_rate, autoplay=True, normalize=False)
